<a href="https://colab.research.google.com/github/sivasaiyadav8143/Machine-Learning-with-Python/blob/master/Missing_Values_Treatment_with_Predictie_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Missing values**, occur when no data value is stored for the variable in an observation.Missing values in the data is a very common problem in Machine Learning because Real-world data often has missing values.There are many reasons for missing values in the data, some are
1. observations that were not recorded (During a Survey, when participants refuse, or do not know the answer to or accidentally skip an item.)
2. data corruption (errors in computer data that occur during writing, reading, storage, transmission, or processing)

Handling missing data is important as many machine learning algorithms do not support data with missing values.

There are different ways to deal with missing data<br>

1. Mark Missing Values
2. Remove Rows/columns With Missing Values
3. Impute with mean, median or mode
4. **Impute with Algorithms.**

In this notebook, lets see how different algorithms help to estimate missing vales.  

In [10]:
import pandas as pd , numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
pima = pd.read_csv('diabetes.csv')
pima.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
pima.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Seems we have no missing values,thats a good news but lets dig a bit more with describe and see if we can find something strange.

In [8]:
pima.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


Above we can see that for the columns<br>


1.   Glucose
2.   BloodPressure
3.   SkinThickness
4.   Insulin
5.   BMI (Body mass index)

have a minimum value of zero (0). On these columns, a value of zero does not make sense and thus indicates missing value.

In [12]:
cols = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
pima_copy = pima.copy(deep=True)
pima_copy[cols] = pima_copy[cols].replace(0,np.nan)

In [13]:
pima_copy.isna().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [26]:
X = pima_copy.iloc[:,:-1]
y = pima_copy.iloc[:,-1]
X.shape,y.shape

((768, 8), (768,))

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=14)

## Impute with Algorithms.
We will see two popular Algorithm to impute missing values
1. KNNImputer (k-Nearest Neighbors imputation)
2. MissForest (Random Forest imputation)

scikit-learn supports KNNImputer but we will use a new library called **missingpy** which supports both **KNNImputer & MissForest**

**Installation** : pip install missingpy

In [24]:
!pip install missingpy

     |████████████████████████████████| 51kB 1.5MB/s 


In [32]:
from missingpy import KNNImputer
imputer = KNNImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [33]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train_imputed)
X_test_scaled = sc.transform(X_test_imputed)

In [34]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_scaled,y_train)
print('Training Score :',lr.score(X_train_scaled,y_train))
print('Testing Score :',lr.score(X_test_scaled,y_test))

Training Score : 0.7746741154562383
Testing Score : 0.7575757575757576


In [46]:
from missingpy import MissForest
miss = MissForest(max_iter=50,max_depth=3)
X_train_imputed_ = miss.fit_transform(X_train)
X_test_imputed_ = miss.transform(X_test)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


In [47]:
sc_tree = StandardScaler()
X_train_scaled_ = sc_tree.fit_transform(X_train_imputed_)
X_test_scaled_ = sc_tree.transform(X_test_imputed_)

In [48]:
lr.fit(X_train_scaled_,y_train)
print('Training Score :',lr.score(X_train_scaled_,y_train))
print('Testing Score :',lr.score(X_test_scaled_,y_test))

Training Score : 0.7783985102420856
Testing Score : 0.7705627705627706


Advantages of MissForest:
1. It can be applied to mixed data types, numerical and categorical. Using KNN-Impute on categorical data requires it to be first converted into some numerical measure. 
2. It is robust to noisy data and multicollinearity.
3. It can work with high-dimensional data.

Drawbacks:<br>
1. if the dataset is sufficiently small it may be more expensive to run MissForest.
2. it’s an algorithm, not a model object; this means it must be run every time data is imputed, which may not work in some production environments.

# References
1. [missingpy Documentation](https://github.com/epsilon-machine/missingpy)
2. https://towardsdatascience.com/missforest-the-best-missing-data-imputation-algorithm-4d01182aed3